# CJE Calibration Demo

This notebook demonstrates how to use CJE (Causal Judge Evaluation) to calibrate AI survey responses against human ground truth.

**Key Benefits:**
- Get valid estimates using only 5-10% human labels
- Proper uncertainty quantification with confidence intervals
- Statistical comparison between models/policies

## Installation

```bash
pip install edsl[cje]
```

In [ ]:
# For development (run from repo root): pip install -e ".[cje]"
# For production: pip install "edsl[cje]"

# If running from docs/notebooks/, install the local package:
%pip install -e ../.. --quiet

In [ ]:
# Verify edsl is installed from the local repo
import edsl
print(f"EDSL version: {edsl.__version__}")
print(f"Location: {edsl.__file__}")

## Step 1: Run a Survey with Multiple Models

First, we create a simple sentiment survey and run it with two different models.

In [ ]:
from edsl import Survey, QuestionLinearScale, Model, Agent

# Create a simple sentiment question
q = QuestionLinearScale(
    question_name="sentiment",
    question_text="Rate the sentiment of this text: '{{ text }}'",
    question_options=[1, 2, 3, 4, 5],
    option_labels={1: "Very Negative", 5: "Very Positive"}
)

survey = Survey([q])

In [ ]:
from edsl import ScenarioList

# Sample texts to analyze
texts = [
    "I love this product! It's amazing.",
    "This is okay, nothing special.",
    "Terrible experience, would not recommend.",
    "Pretty good overall, minor issues.",
    "Absolutely fantastic, exceeded expectations!",
    "Meh, it's fine I guess.",
    "Worst purchase ever.",
    "Great quality, fast shipping.",
    "Not worth the money.",
    "Highly recommend to everyone!",
]

scenarios = ScenarioList.from_list("text", texts)

In [ ]:
# Run with two models
models = [Model("gpt-4o"), Model("claude-sonnet-4-20250514")]

results = survey.by(scenarios).by(models).run()

In [ ]:
# View results
results.select("model", "text", "sentiment")

## Step 2: Add Human Labels (Oracle)

In practice, you would collect human ratings for a sample of responses.
Here we simulate this with "ground truth" ratings for ~50% of samples.

In [ ]:
import random

# Simulate human labels for some samples
# In practice, you would collect these from actual human raters
human_labels = []

for i, result in enumerate(results):
    # Add human label for ~50% of samples
    if random.random() < 0.5:
        # Simulate a human rating (1-5 scale)
        text = result["scenario"]["text"]
        if "love" in text or "amazing" in text or "fantastic" in text:
            human_labels.append(5)
        elif "terrible" in text or "worst" in text:
            human_labels.append(1)
        elif "good" in text or "great" in text:
            human_labels.append(4)
        elif "okay" in text or "fine" in text or "meh" in text:
            human_labels.append(3)
        else:
            human_labels.append(3)
    else:
        human_labels.append(None)  # No human label for this sample

print(f"Human labels collected: {sum(1 for x in human_labels if x is not None)}/{len(human_labels)}")

## Step 3: Calibrate with CJE

Now we use CJE to calibrate the AI responses against human labels.

In [ ]:
# Transform 1-5 scale to 0-1 for CJE
scale_transform = lambda x: (x - 1) / 4

# Calibrate
cal_result = results.calibrate(
    question_name="sentiment",
    oracle_labels=human_labels,
    policy_column="model",
    score_transform=scale_transform,
    verbose=True,
)

In [ ]:
# View calibrated estimates and forest plot
print(cal_result)
cal_result.plot();

## Step 4: Compare Models

Use statistical testing to compare model performance.

In [ ]:
# Get the best performing model
print(f"Best model: {cal_result.best_policy()}")
print(f"Ranking: {cal_result.ranking()}")

In [ ]:
# Statistical comparison between models
policies = list(cal_result.estimates.keys())
if len(policies) >= 2:
    comparison = cal_result.compare(policies[0], policies[1])
    print(f"\nComparison: {policies[0]} vs {policies[1]}")
    print(f"Difference: {comparison.difference:.3f}")
    print(f"p-value: {comparison.p_value:.3f}")
    print(f"Significant at α=0.05: {comparison.significant}")

## Summary

CJE calibration provides:

1. **Efficient use of human labels** - Only need 5-10% oracle labels to calibrate
2. **Valid point estimates** - Corrects for systematic biases in AI judge scores
3. **Uncertainty quantification** - Confidence intervals for each policy
4. **Statistical comparisons** - Proper hypothesis testing between policies

### Next Steps

- Collect real human labels for your survey responses
- Use `CalibrationResult.compare()` to make data-driven model selection decisions
- Monitor for drift by re-calibrating when models are updated